In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

import statsmodels as sm
import sklearn.preprocessing as preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy import stats

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
gasoline_proc = pd.read_csv('../../data/03_processed/gasoline_processed.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,14,15,16,19,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
gasoline_proc['datesampled'] = pd.to_datetime(gasoline_proc['datesampled'])

In [31]:
gasoline_proc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19399 entries, 0 to 19398
Data columns (total 38 columns):
Sample                          19399 non-null object
prod                            19399 non-null object
datesampled                     19399 non-null datetime64[ns]
grade                           19399 non-null object
supplier                        16344 non-null object
facilityname                    19399 non-null object
siteaddress                     19250 non-null object
units_dist_50                   19399 non-null object
units_vap_pressure              19399 non-null object
units_vap_liq_pressure          19399 non-null object
method_dist_50                  19399 non-null object
method_vap_pressure             19399 non-null object
method_vap_liq_pressure         19399 non-null object
result_dist_50                  19399 non-null object
result_vap_pressure             19399 non-null object
result_vap_liq_pressure         19399 non-null object
minresults_dist_50 

In [27]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
ax = sns.heatmap(gasoline_proc)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

### Models - Compliance Distilation 50% 

Possible predictor variables

1. TN_retailers_seasons
1. grade
1. supplier
1. siteaddress

Grade vs. compliance dist 50

In [15]:
gasoline_proc.columns

Index(['Sample', 'prod', 'datesampled', 'grade', 'supplier', 'facilityname',
       'siteaddress', 'units_dist_50', 'units_vap_pressure',
       'units_vap_liq_pressure', 'method_dist_50', 'method_vap_pressure',
       'method_vap_liq_pressure', 'result_dist_50', 'result_vap_pressure',
       'result_vap_liq_pressure', 'minresults_dist_50',
       'minresults_vap_pressure', 'minresults_vap_liq_pressure',
       'maxresults_dist_50', 'maxresults_vap_pressure',
       'maxresults_vap_liq_pressure', 'compliance_dist_50',
       'compliance_vap_pressure', 'compliance_vap_liq_pressure', 'zipcode',
       'siteaddress_city', 'Date', 'TN_retailers_seasons',
       'TN_distributor_seasons', 'vapor_liquid_minC_retail',
       'distillation_50_minC _retail', 'distillation_50_maxC_retail',
       'vapor_pressure_maxC_retail', 'vapor_liquid_minC_dist',
       'distillation_50_minC _dist', 'distillation_50_maxC_dist',
       'vapor_pressure_maxC-dist'],
      dtype='object')

In [22]:
# construct features 
x_feats = ['TN_retailers_seasons', 'grade']
X = pd.get_dummies(gasoline_proc[x_feats], dtype=float)
# convert target using get_dummies
y = pd.get_dummies(gasoline_proc["compliance_dist_50"], drop_first=True, dtype=float)

In [23]:
import statsmodels.api as sm

X = sm.add_constant(X)
# fit model
logit_model = sm.Logit(y.iloc[:,0], X)
# get results of the fit
result = logit_model.fit()

         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

In [21]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      Y   No. Observations:                19399
Model:                          Logit   Df Residuals:                    19389
Method:                           MLE   Df Model:                            9
Date:                Tue, 30 Jul 2019   Pseudo R-squ.:                 0.09640
Time:                        10:17:34   Log-Likelihood:                -18.397
converged:                      False   LL-Null:                       -20.360
                                        LLR p-value:                    0.9163
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                            18.0681   2420.208      0.007      0.994   -4725.452    4761.588
TN_retailers_seasons_ A-2/C-3     8.6070   1.92e+04      0.000      1.000   -3.76e+04    3.76e+04
TN_retailers_seasons_ A-3/C-3     1.3824   2449.373      0.001      1.000   -4799.301    4802.066
TN_retailers_seasons_ C-3/D-4    -9.8394   2420.208     -0.004      0.997   -4753.359    4733.681
TN_retailers_seasons_ D-4        -0.3753   2423.884     -0.000      1.000   -4751.100    4750.349
TN_retailers_seasons_ D-4/A-3     3.7472   2703.679      0.001      0.999   -5295.367    5302.861
TN_retailers_seasons_ E-5/D-4     4.6647   3501.529      0.001      0.999   -6858.207    6867.536
TN_retailers_seasons_A-2        -10.0126   2420.208     -0.004      0.997   -4753.533    4733.507
TN_retailers_seasons_A-3         -8.1721   2420.209     -0.003      0.997   -4751.695    4735.351
TN_retailers_seasons_D-4          9.7381   3.05e+04      0.000      1.000   -5.97e+04    5.97e+04
=================================================================================================

Possibly complete quasi-separation: A fraction 0.64 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

Supplier vs. compliance dist 50

In [ ]:
# construct features 
x_feats = ['supplier']
X = pd.get_dummies(gasoline_proc[x_feats], dtype=float)
# convert target using get_dummies
y = pd.get_dummies(gasoline_proc["compliance_dist_50"], drop_first=True, dtype=float)

In [ ]:
import statsmodels.api as sm

X = sm.add_constant(X)
# fit model
logit_model = sm.Logit(y.iloc[:,0], X)
# get results of the fit
result = logit_model.fit()

In [ ]:
result.summary()